In [2]:
k = 100;
value = 0;
for i=1:k
    value = value + 1/(i^2);
end

@show typeof(value)

typeof(value) = Float64


Float64

In [123]:
#import Pkg
#Pkg.add("BenchmarkTools")
using BenchmarkTools
using LinearAlgebra
using SparseArrays

function u_real(x,alpha)
    value = -1/2 * x^2 + 1/2 * x + 1/(2 * alpha);
    return value
end

function Progonka(A,b)
    N = length(b);
    alpha = ones(N);
    beta = ones(N);
    u = ones(N);
    alpha[2] =  - A[1,2] / A[1,1];
    beta[2] = b[1] / A[1,1];
    
    for i = 2:N-1
        alpha[i+1] = - A[i,i+1] / (A[i,i-1] * alpha[i] + A[i,i]);
        beta[i+1] = (b[i] - A[i,i-1] * beta[i]) / (A[i,i-1] * alpha[i] + A[i,i]);
    end
    
    u[N] = (b[N] - A[N,N-1] * beta[N]) / (A[N,N-1] * alpha[N] + A[N,N]);
    
    for i = N-1:-1:1
        u[i] = alpha[i+1] * u[i+1] + beta[i+1];
    end
    
    return u;
end

k = 6;
N = 2^k+1;
alpha = 1;
v_L = 0;
v_R = 0;
f = 1;
h = 1/(N-1);
dl = ones(N-1);
d = ones(N);
b = ones(N);
u = ones(N);
x = ones(N);
u_r = ones(N);
error = ones(N);
average = 0;

for i = 1:N
    x[i] = (i-1) * h;
end

d = d * 2/h;
dl = dl * (-1/h);
d[1] = alpha + 1/h;
d[N] = alpha + 1/h;

b = b * f * h;
b[1] = alpha * v_L;
b[N] = alpha * v_R;

A_tri = Tridiagonal(dl,d,dl);

#u = A\b;

#u = Progonka(A_tri,b);

#F = lu(A_tri);
#y = F.L\b;
#u = F.U\y;

#A_full = Matrix(A_tri);
#F = lu(A_full);
#y = F.L\b;
#u = F.U\y;

A_sparse = sparse(A_full);
F = lu(A_sparse);
y = F.L\b;
u = F.U\y;

#A_inv = inv(A_tri);
#@show A_inv
#u = A_inv * b;

#nonzerocounter = 0;
#for i = 1:N
#    for j = 1:N
#        if A_inv[i,j] != 0
#            nonzerocounter = nonzerocounter + 1;
#        end
#    end
#end

#@show nonzerocounter
#@show N*N




for i = 1:N
    u_r[i] = u_real(x[i],alpha);
    error[i] = abs(u[i]-u_r[i]);
    average = average + error[i];
end

average = average / N;



@show average
#@benchmark Progonka(A_tri,b)
#@benchmark lu(A_tri)




average = 145.3954927884592


145.3954927884592

In [108]:
benchmark

UndefVarError: UndefVarError: benchmark not defined